In [1]:
import pandas as pd
import scholarly
from collections import defaultdict
import re
import pickle

In [2]:
# Please change this to the year you are 
global_year = 2018

## Read dataset

In [3]:
data_raw = pd.read_pickle('../features/all_data_features_17_20.pkl')
# data_raw.head()
data = data_raw[["id", "title", "label", "authors"]]
org_papers = data.to_dict('index')

### Segregate authors based on year

In [51]:
yearwise_authors = {y: set() for y in range(2017, 2021)}

for p in org_papers:
    year = int(p.split("_")[0])
    yearwise_authors[year].add(org_papers[p]["authors"][-1])
#     yearwise_authors[year].update(set(org_papers[p]["authors"][-2:]))
#     for auth in org_papers[p]["authors"]:
#         yearwise_authors[year].add(auth)

In [52]:
org_papers[p]["authors"][-1]

'Eric P. Xing'

In [54]:
len(yearwise_authors[global_year])#, yearwise_authors[global_year]

649

### Scrape GS author info

In [ ]:
# RUN THIS ONLY THE FIRST TIME
author_info = {}
err_authors = []

with open("author_info_scholar_2018.pickle", "wb") as f:
    pickle.dump(author_info, f)

In [8]:
p = re.compile(" [A-Z]\.? ")

In [9]:
# RESTART FROM HERE AFTER COLAB DISCONNECTS

with open("author_info_scholar_2018.pickle", "rb") as f:
    author_info = pickle.load(f)

count = len(author_info)
done_authors = list(author_info.keys())

In [62]:
count, len(err_authors), len(yearwise_authors[global_year])

(518, 10, 649)

In [66]:
with open("author_info_scholar_2018.pickle", "rb") as f:
    old_author_info = pickle.load(f)
old_author_info.update(author_info)
with open("author_info_scholar_2018.pickle", "wb") as f:
    pickle.dump(old_author_info, f)

print(len(old_author_info))

658


In [68]:
len(yearwise_authors[global_year].difference(set(list(old_author_info.keys()))))

330

In [ ]:
for a in yearwise_authors[global_year]:
    
    if not a in done_authors and not a in err_authors:
        
        if count % 10 == 0:
            print("Status {} / {}".format(count, len(yearwise_authors[global_year])))
            with open("author_info_scholar_2018.pickle", "rb") as f:
                old_author_info = pickle.load(f)
            old_author_info.update(author_info)
            with open("author_info_scholar_2018.pickle", "wb") as f:
                pickle.dump(old_author_info, f)
            author_info = {}
            old_author_info = {}
            
        auth_candidates = []
        try:
            print(a)
            c = scholarly.search_author(a)
            fill_count = 0
            for i in c:
                if fill_count > 6:
                    break
                auth_candidates.append((i, i.fill()))
                fill_count += 1

            if not auth_candidates:
                # no author found
                a_clean = p.sub(" ", a)
                c = scholarly.search_author(a_clean)
                fill_count = 0
                for i in c:
                    if fill_count > 6:
                        break
                    auth_candidates.append((i, i.fill()))
                    fill_count += 1

            author_info[a] = auth_candidates
            done_authors.append(a)
        except Exception as ex:
            print("Exception: ", a)
            err_authors.append(a)
        count += 1

Bin Yu
Cane Punma
Exception:  Cane Punma
Javad Sadri
David Z. Pan
Alex Nevidomsky
Exception:  Alex Nevidomsky
David Tse
Ioannis Mitliagkas
and Jan Kleindienst
Exception:  and Jan Kleindienst
Status 700 / 649
Kevin Murphy
Frank Hutter
Daniela Rus
Hai Li
Rujie Liu
Kilian Weinberger
Balaraman Ravindran
Yutaka Matsuo
Sumit Gulwani
Joonseok Lee
Status 710 / 649
Franz Pernkopf
Yangang Zhang
Exception:  Yangang Zhang
Jian Zhang
Ryuki Tachibana
Sameer Singh
Gerald Tesauro
Fred Bertsch
Yu Qian
Said Ladjal
Yoshua Bengio
Status 720 / 649
Virginia R. de Sa
Exception:  Virginia R. de Sa
Avinash Balakrishnan
Nick Johnston
Weizhu Chen
Martine De Cock
Zhouyuan Li
Xue-Xin Wei
Laura Balzano
Mehran Mesbahi
Peter I. Frazier
Status 730 / 649
Aaron Courville
Jaakko Lehtinen
Soummya Kar
Exception:  Soummya Kar
Sumit Baburao Tamgale
Exception:  Sumit Baburao Tamgale
Honglak Lee
Teresa Ludermir


In [64]:
len(yearwise_authors[2018])

649

In [65]:
len(author_info)

2

### Save info

In [ ]:
# with open("author_info_scholar_2018.pickle", "wb") as f:
#     pickle.dump(author_info, f)

In [11]:
cp author_info_scholar_2018.pickle bkp_author_info_scholar_2018.pickle

## Disambiguate authors

### 1. Read gs scrapped data

In [ ]:
with open("author_info_scholar_2018.pickle", "rb") as f:
    all_authors_gs = pickle.load(f)

### 2. Read conflicts data

In [30]:
%%bash
ls ../../ICLR\ data/masterdata_unbalanced/

ICLR data-20200322T071325Z-001.zip
off_rev_dict_2017.pkl
off_rev_dict_2018.pkl
off_rev_dict_2019.pkl
off_rev_dict_2020.pkl
paper_decision_dict_2017.pkl
paper_decision_dict_2018.pkl
paper_decision_dict_2019.pkl
paper_decision_dict_2020.pkl
papers_2017.pkl
papers_2018.pkl
papers_2019.pkl
papers_2020.pkl
readme


In [34]:
data_path = "../../ICLR data/masterdata_unbalanced/"

years = [2019]

for y in years:
    papers_data = pd.read_pickle(data_path + "papers_{}.pkl".format(y))

In [35]:
conflicts_dict = {}
emails_dict = {}

for p in papers_data:
    conflicts_dict[p] = papers_data[p]["content"]["conflicts"]
    emails_dict[p] = papers_data[p]["content"]["authorids"]

KeyError: 'conflicts'

In [37]:
papers_data[p]["content"]

{'TL;DR': 'We provide improved upper bounds for the number of linear regions used in network expressivity, and an highly efficient algorithm (w.r.t. exact counting) to obtain probabilistic lower bounds on the actual number of linear regions.',
 '_bibtex': '@misc{\nserra2019empirical,\ntitle={Empirical Bounds on Linear Regions of Deep Rectifier Networks},\nauthor={Thiago Serra and Srikumar Ramalingam},\nyear={2019},\nurl={https://openreview.net/forum?id=B1MAJhR5YX},\n}',
 'abstract': 'One form of characterizing the expressiveness of a piecewise linear neural network is by the number of linear regions, or pieces, of the function modeled. We have observed substantial progress in this topic through lower and upper bounds on the maximum number of linear regions and a counting procedure. However, these bounds only account for the dimensions of the network and the exact counting may take a prohibitive amount of time, therefore making it infeasible to benchmark the expressiveness of networks. 

### 3. Start disambiguation

In [ ]:
paper_authors_info = defaultdict(dict)
per_paper = defaultdict(list)

still_not_found = 0
total_auth_count = 0
found_count = 0
not_in_gs = 0
cc = 0

still_not_found_list = []

for p in org_papers:
    for auth in org_papers[p]["authors"]:
        total_auth_count += 1
        disambiguated_author = None
        
        if auth in all_authors_gs:
            if len(all_authors_gs[auth]) == 1:
                disambiguated_author = all_authors_gs[auth][0][0]
                found_count += 1
            else:
                found_paper = False
                for mauth in all_authors_gs[auth]:
                    for pub in mauth[0].publications:
                        org_title = ''.join(filter(str.isalpha, org_papers[p]["title"].lower()))
                        gs_pub_title = ''.join(filter(str.isalpha, pub.bib["title"].lower()))
                        if org_title == gs_pub_title:
                            found_paper = True
                            disambiguated_author = mauth[0]
                            found_count += 1
                            break
                    if found_paper:
                        break
                if not found_paper:
                    # Paper not found, check via conflicts/email information
                    matching_affils = []
                    
                    for conflict_affil in conflicts_dict[p]:
                        for mauth in all_authors_gs[auth]:
                            if mauth[0].email.find(conflict_affil) > -1:
                                matching_affils.append(mauth)
                    
                    # still not found
                    if len(matching_affils) == 0:
                        still_not_found += 1
                        still_not_found_list.append(auth)
                    elif len(matching_affils) == 1:
                        disambiguated_author = matching_affils[0][0]
                        found_count += 1
                    else:
                        potential_emailids = []
                        auth_split = auth.split(" ")
                        for splitname in auth_split:
                            for authemailid in emails_dict[p]:
                                if authemailid.find(splitname) > -1:
                                    potential_emailids.append(authemailid)
                                    
                        potential_emailids = list(set(potential_emailids))
                        if len(potential_emailids) == 1:
                            domain = potential_emailids[0].split("@")[-1]
                            identified_auth = []
                            for m in matching_affils:
                                if m[0].email.find(domain) > -1:
                                    identified_auth.append(m[0])
                            
                            if len(identified_auth) == 1:
                                disambiguated_author = m[0]
                                found_count +=1 
                            else:
                                still_not_found += 1
                                still_not_found_list.append(auth)
                        else:
                            still_not_found += 1
                            still_not_found_list.append(auth)
            if not disambiguated_author is None:
                cc +=1
                paper_authors_info[auth] = disambiguated_author
                per_paper[p].append(auth)
        else:
            not_in_gs +=1

In [ ]:
print("Found authors: ", found_count)
print("Added to dict: ", cc)
print("Multiple entries in GS but no publication with the same title found or empty GS: ", still_not_found)
print("NOt in GS data: ", not_in_gs)
print("Total authors to be disambiguated: ", total_auth_count)

## Save citation count, publication count, and hindex

In [ ]:
for k, v in org_papers.items():
    org_papers[k]["pub_count_gs"] = []
    org_papers[k]["cit_count_gs"] = []
    org_papers[k]["hindex"] = []
    
    for a in v["authors"]:
        if a in paper_authors_info:
            temp_auth_dict = paper_authors_info[a].__dict__
            if "citedby" in temp_auth_dict and temp_auth_dict["citedby"] > 0:
                total_citations = temp_auth_dict["citedby"]
                if "cites_per_year" in temp_auth_dict:
                    for y in temp_auth_dict["cites_per_year"]:
                        if y > (global_year-1):
                            total_citations -= temp_auth_dict["cites_per_year"][y]
                org_papers[k]["cit_count_gs"].append(total_citations)
            if "hindex" in temp_auth_dict:
                if "hindex5y" in temp_auth_dict:
                    org_papers[k]["hindex"].append((temp_auth_dict["hindex5y"] + temp_auth_dict["hindex"])/2)
                else:
                    org_papers[k]["hindex"].append(temp_auth_dict["hindex"])
            if "publications" in temp_auth_dict:
                total_publications = 0
                for pub in temp_auth_dict["publications"]:
                    if "year" in pub.__dict__["bib"]:
                        if pub.__dict__["bib"]["year"] < global_year:
                            total_publications += 1
                    else:
                        total_publications += 1
                org_papers[k]["pub_count_gs"].append(total_publications)

In [ ]:
# Save file
